# Decision Trees and Random Forest Code Along.

We'll work through all 3 three methods discussed and compare their results on a college dataset.

The dataset has features of universities and labeled either Private or Public.

The feature definitions are as follows:
- ```Private```: A factor with levels No and Yes indicating private or public university
- ```Apps```: Number of applications received
- ```Accept```: Number of applications accepted
- ```Enroll```: Number of new students enrolled
- ```Top10perc```: Pct. new students from top 10% of H.S. class
- ```Top25perc```: Pct. new students from top 25% of H.S. class
- ```F_Undergrad```: Number of fulltime undergraduates
- ```P_Undergrad```: Number of parttime undergraduates
- ```Outstate```: Out-of-state tuition
- ```Room_Board```: Room and board costs
- ```Books```: Estimated book costs
- ```Personal```: Estimated personal spending
- ```PhD```: Pct. of faculty with Ph.D.’s
- ```Terminal```: Pct. of faculty with terminal degree
- ```S_F_Ratio```: Student/faculty ratio
- ```perc_alumni```: Pct. alumni who donate
- ```Expend```: Instructional expenditure per student
- ```Grad_Rate```: Graduation rate

In [1]:
# Start a spark Session
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("tree").getOrCreate()

In [3]:
data = spark.read.csv("College.csv", inferSchema=True, header=True)

In [4]:
data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [5]:
# E.D.A.
data.head(1)[0]

Row(School='Abilene Christian University', Private='Yes', Apps=1660, Accept=1232, Enroll=721, Top10perc=23, Top25perc=52, F_Undergrad=2885, P_Undergrad=537, Outstate=7440, Room_Board=3300, Books=450, Personal=2200, PhD=70, Terminal=78, S_F_Ratio=18.1, perc_alumni=12, Expend=7041, Grad_Rate=60)

### Proceed by formatting the data

In [6]:
from pyspark.ml.feature import VectorAssembler

In [7]:
data.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [8]:
assembler = VectorAssembler(inputCols=['Apps', 'Accept', 'Enroll', 'Top10perc', 'Top25perc',
                                       'F_Undergrad', 'P_Undergrad', 'Outstate', 'Room_Board',
                                       'Books', 'Personal', 'PhD', 'Terminal', 'S_F_Ratio',
                                       'perc_alumni', 'Expend', 'Grad_Rate'],
                            outputCol="features")

In [9]:
data_transformed = assembler.transform(data)

In [10]:
# The "Private" column contains either "yes" or "no".
# Wee need to convert this to binary format: 0, 1.

In [11]:
from pyspark.ml.feature import StringIndexer

In [12]:
indexer = StringIndexer(inputCol="Private", outputCol="PrivateIndex")

In [13]:
data_indexed = indexer.fit(dataset=data_transformed).transform(dataset=data_transformed)
data_indexed

DataFrame[School: string, Private: string, Apps: int, Accept: int, Enroll: int, Top10perc: int, Top25perc: int, F_Undergrad: int, P_Undergrad: int, Outstate: int, Room_Board: int, Books: int, Personal: int, PhD: int, Terminal: int, S_F_Ratio: double, perc_alumni: int, Expend: int, Grad_Rate: int, features: vector, PrivateIndex: double]

In [14]:
data_indexed.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = false)



In [15]:
data_final = data_indexed.select("features", "PrivateIndex")

In [16]:
train_data, test_data = data_final.randomSplit([0.7, 0.3], seed=123)

### Perform (tree-method) classification on the data.

In [17]:
from pyspark.ml.classification import (DecisionTreeClassifier, 
                                       GBTClassifier, RandomForestClassifier)

In [18]:
dtc = DecisionTreeClassifier(labelCol="PrivateIndex", featuresCol="features", maxDepth=8)
rfc = RandomForestClassifier(labelCol="PrivateIndex", featuresCol="features", numTrees=500, maxDepth=8)
gbt = GBTClassifier(labelCol="PrivateIndex", featuresCol="features", maxDepth=8)

In [19]:
# Train all of the models
dtc_fitted = dtc.fit(train_data)
rfc_fitted = rfc.fit(train_data)
gbt_fitted = gbt.fit(train_data)

In [20]:
dtc_preds = dtc_fitted.transform(test_data)
rfc_preds = rfc_fitted.transform(test_data)
gbt_preds = gbt_fitted.transform(test_data)

In [21]:
# Let us get some evaluation metrics and see how our different models performed:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [22]:
bin_evaluator = BinaryClassificationEvaluator(labelCol="PrivateIndex")

In [23]:
bin_evaluator.evaluate(dataset=dtc_preds)
# This by itself returns the auroc metric!

0.915819209039548

In [24]:
print("DTC ---> {:6.4f}.".format(bin_evaluator.evaluate(dataset=dtc_preds)))
print("RFC ---> {:6.4f}.".format(bin_evaluator.evaluate(dataset=rfc_preds)))
print("GBT ---> {:6.4f}.".format(bin_evaluator.evaluate(dataset=gbt_preds)))

DTC ---> 0.9158.
RFC ---> 0.9752.
GBT ---> 0.9430.


### Explore other Metrics by using the ```MulticlassClassificationEvaluator```.

In [25]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [26]:
# Let us say, for example, that this is an accuracy evaluator:
acc_evaluator = MulticlassClassificationEvaluator(labelCol="PrivateIndex",
                                                  metricName="accuracy")

In [27]:
print("DTC ---> {:6.4f}.".format(acc_evaluator.evaluate(dataset=dtc_preds)))
print("RFC ---> {:6.4f}.".format(acc_evaluator.evaluate(dataset=rfc_preds)))
print("GBT ---> {:6.4f}.".format(acc_evaluator.evaluate(dataset=gbt_preds)))

DTC ---> 0.9256.
RFC ---> 0.9380.
GBT ---> 0.9256.
